# Alpaca Trading Workbook

### Data Scraper

In [69]:
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame, TimeFrameUnit
import sqlite3
import pandas as pd
import ta
from datetime import datetime, timedelta

class AlpacaTradingBot:
    def __init__(self, keys_file_path='alpaca_keys.txt', base_url='https://paper-api.alpaca.markets', database_path=r'D:\Scripts\alpaca\alpaca_algo_trading\alpaca_data.db'):
        with open(keys_file_path, 'r') as file:
            self.api_key = file.readline().strip()
            self.api_secret = file.readline().strip()
        self.base_url = base_url
        self.api = tradeapi.REST(self.api_key, self.api_secret, base_url=base_url)
        self.database_path = database_path


    def download_bar_data(self, stock, timeframe, start_date, end_date):
        all_data = []

        current_date = start_date
        while current_date <= end_date:
            # Get data for the current day
            bars = self.api.get_bars(stock, timeframe, start=current_date, end=current_date, limit=1000).df
            if bars.empty:
                continue

            bars['symbol'] = stock  # Add the stock symbol column
            all_data.append(bars)

            # Move to the next day
            current_date = (datetime.strptime(current_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        if all_data:
            combined_data = pd.concat(all_data)
            combined_data = combined_data.reset_index()  # Ensure the index is reset to have 'timestamp' as a column
            return combined_data[['symbol'] + [col for col in combined_data.columns if col != 'symbol']]
        else:
            return pd.DataFrame()

    # This method calculates various indicators on a dataframe
    def calculate_indicators(self, stock_data):
        stock_data['rsi'] = ta.momentum.rsi(stock_data['close'], window=14)
        stock_data['sma_50'] = ta.trend.sma_indicator(stock_data['close'], window=50)
        stock_data['sma_200'] = ta.trend.sma_indicator(stock_data['close'], window=200)
        stock_data['bollinger_hband'] = ta.volatility.bollinger_hband(stock_data['close'], window=20, window_dev=2)
        stock_data['bollinger_lband'] = ta.volatility.bollinger_lband(stock_data['close'], window=20, window_dev=2)
        return stock_data

    # This method executes a SQL statement against our database.
    def db_write(self, sql_statement):
        db_path = self.database_path

        # Connect to the database & create a cursor object
        conn = sqlite3.connect(db_path)
        cur = conn.cursor()

        # Execute the SQL statement
        cur.execute(sql_statement)

        # Commit the changes & close the connection
        conn.commit()
        conn.close()

    # This method appends records to our database.
    def db_append(self, table_name, data_frame):
        db_path = self.database_path

        # Connect to the database
        conn = sqlite3.connect(db_path)

        # Append our dataframe into our table
        data_frame.to_sql(table_name, conn, schema='main', if_exists='append', index=False)

        # Commit the changes & close the connection
        conn.commit()
        conn.close()

    # Modification of db_append that checks primary key & ensures we aren't inserting a duplicate value
    def db_append_no_duplicates(self, table_name, data_frame):
        db_path = self.database_path

        # Connect to the database
        conn = sqlite3.connect(db_path)

        # Get existing primary keys from the table
        existing_keys_query = f"SELECT PRIMARY_KEY FROM {table_name}"
        existing_keys = pd.read_sql(existing_keys_query, conn)['PRIMARY_KEY'].tolist()

        # Filter out rows with primary keys that already exist in the table
        data_frame_new_records = data_frame[~data_frame['PRIMARY_KEY'].isin(existing_keys)]

        # Append only the new rows into our table
        data_frame_new_records.to_sql(table_name, conn, schema='main', if_exists='append', index=False)

        # Commit the changes & close the connection
        conn.commit()
        conn.close()


### Schema DDL

In [66]:
#Schema DDL
atb = AlpacaTradingBot()

# This table will house the raw schedule
sql_statement = atb.db_write('''DROP TABLE IF EXISTS STG_SYMBOL_DATA''')
sql_statement = atb.db_write('''CREATE TABLE IF NOT EXISTS STG_SYMBOL_DATA (
    SYMBOL                  TEXT        PRIMARY_KEY,
    TIMESTAMP               TIMESTAMP   PRIMARY_KEY,
    CLOSE                   DECIMAL,
    HIGH                    DECIMAL,
    LOW                     DECIMAL,
    TRADE_COUNT             INTEGER,
    OPEN                    DECIMAL,
    VOLUME                  INTEGER,
    VWAP                    DECIMAL
)
''')

### Calling the class

In [70]:
atb = AlpacaTradingBot()

# Define the list of stocks
stocks = ["SPY"]

# Define the time range for historical data
start_date = "2020-01-01"
end_date = "2020-12-31"
# end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data
data = {}
for stock in stocks:
    stock_data = atb.download_bar_data(stock, TimeFrame(5, TimeFrameUnit.Minute), start_date, end_date)
    data[stock] = stock_data

# Store data locally
for stock, stock_data in data.items():
    atb.db_append_no_duplicates('STG_SYMBOL_DATA', stock_data)

sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/SPY/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/